<a href="https://colab.research.google.com/github/Howl06/practice/blob/main/new_bot_002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

# 安裝套件
!pip install line-bot-sdk flask flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 9.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1


In [3]:
# web server
from flask import Flask, request, abort, jsonify
import json
from flask_ngrok import run_with_ngrok
from linebot import(
    LineBotApi,WebhookHandler
)
# 引用無效簽章錯誤
from linebot.exceptions import(
    InvalidSignatureError
)

In [4]:
# 建置handler 與 line_bot_api
app = Flask(__name__,static_url_path = "/material", static_folder = "./material/")
run_with_ngrok(app)
#生成實體物件
line_bot_api = LineBotApi("vtaRo4pkLqIZjk50gBJ53fFtMpg+8unl4TvLKbyvENmbWkzdriMNIeVm87ZWRBdTeDL7YM4hGte+BrNMmYscJ1WNPvCQbheXUbVevrlKB3zP3CaeRjbPQRPHMsqW+037bLOogqjOrh5ssRj53kqQMQdB04t89/1O/w1cDnyilFU=")
handler = WebhookHandler("de385af73d2121e65ca6fc0490991b02")

In [5]:
"""
建置主程序API入口 接收line傳來消息
並取出消息內容
將消息內容存在google drive的檔案內
並請handler 進行消息驗證與轉發
"""
@app.route("/",methods=["POST"])
def callback():
    # get X-Line-Signature header value
    signature = request.headers["X-Line-Signature"]

    # get request body as text
    body = request.get_data(as_text=True)
    print(body)

    #紀錄用戶 log
    with open("/content/drive/MyDrive/ai-event.log", "a") as f:
        f.write(body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return "OK"

In [13]:
"""
撰寫用戶關注時,我們要處理的商業邏輯
1.取得用戶個資,並存回伺服器
2.回應用戶,歡迎用的文字消息
"""


# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)

from linebot.models import(
    TextSendMessage, ImageSendMessage
)

# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
     # 將用戶資訊存在檔案內
    with open("/content/drive/MyDrive/users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\n')
    
    # 回覆文字消息與圖片消息
    line_bot_api.reply_message(
        event.reply_token,
        [TextSendMessage('安安，你的個資已被我紀錄了')]
    )

In [14]:
# 運行主程序
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://4cdf-34-125-100-47.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [16/Mar/2023 03:39:32] "POST / HTTP/1.1" 200 -


{"destination":"U54dcf9ab5f592860405fcec0d6979997","events":[{"type":"unfollow","webhookEventId":"01GVM80QNQ9K7C1WYWB709GR28","deliveryContext":{"isRedelivery":false},"timestamp":1678937972291,"source":{"type":"user","userId":"Ua19163cfb927dce013b280948e91caf0"},"mode":"active"}]}
{"destination":"U54dcf9ab5f592860405fcec0d6979997","events":[{"type":"follow","webhookEventId":"01GVM80RTJ48G614HBEEQ5XVGX","deliveryContext":{"isRedelivery":false},"timestamp":1678937973253,"source":{"type":"user","userId":"Ua19163cfb927dce013b280948e91caf0"},"replyToken":"a86da190c97c44ffa7d9e838a10caa32","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [16/Mar/2023 03:39:35] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Mar/2023 03:39:42] "POST / HTTP/1.1" 200 -


{"destination":"U54dcf9ab5f592860405fcec0d6979997","events":[{"type":"message","message":{"type":"text","id":"17810338149536","text":"終於\n"},"webhookEventId":"01GVM810YCK40152WHVVRT0N5R","deliveryContext":{"isRedelivery":false},"timestamp":1678937981834,"source":{"type":"user","userId":"Ua19163cfb927dce013b280948e91caf0"},"replyToken":"5104c12f85f84ba2a54ab49d5bfff720","mode":"active"}]}
